#### Lab 02 - Collaborative Filtering (User, Film, Rating)

In [1]:
import warnings, pandas

warnings.filterwarnings("ignore")

In [2]:
rating = pandas.read_table("ratings.csv", sep=",")

rating.iloc[:5]

,UserID,MovieID,Rating,Timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [3]:
rating.tail()

,UserID,MovieID,Rating,Timestamp
100831,610,166534,4.0,1493848402
100832,610,168248,5.0,1493850091
100833,610,168250,5.0,1494273047
100834,610,168252,5.0,1493846352
100835,610,170875,3.0,1493846415


In [4]:
# Check Null Value on Rating Table

null = rating.isnull().sum()

null.sum()

0

In [5]:
# Check Average Rating on Rating Table

rata_rata_rating = rating["Rating"].mean()

rata_rata_rating

3.501556983616962

In [6]:
# Get Total Film

# Our Total Film is 9.724 Film

total_film = rating["MovieID"].nunique()

total_film

9724

In [7]:
# Get Total User

# Our Total User is 610 User

total_user = rating["UserID"].nunique()

total_user

610

In [8]:
usecols = ["MovieID", "Title"]

table_film = pandas.read_table("movies.csv", sep=",", usecols=usecols)

table_film.iloc[:5]

,MovieID,Title
0,1,Toy Story (1995)
1,2,Jumanji (1995)
2,3,Grumpier Old Men (1995)
3,4,Waiting to Exhale (1995)
4,5,Father of the Bride Part II (1995)


In [9]:
rating_table = rating.pivot_table(index="MovieID", columns="UserID", values="Rating")

SparseRate = lambda table : round((table.isnull().sum().sum() / table.size) * 100, 3)

SparseRate(rating_table)

98.3

In [10]:
minimum_film_interaction = 10

WatchFilm = rating.groupby("MovieID")["Rating"].agg(["count", "mean"])

WatchFilm.columns = ["TotalWatch", "Rating"]

WatchFilm = WatchFilm[WatchFilm["TotalWatch"] > minimum_film_interaction]

rating = rating[rating["MovieID"].isin(WatchFilm.index)]

rating["MovieID"].nunique()

2121

In [11]:
rating_table = rating.pivot_table(index="MovieID", columns="UserID", values="Rating")

SparseRate(rating_table)

93.845

In [12]:
minimum_user_interaction = 50

UserWatch = rating.groupby("UserID")[["Rating"]].count()

UserWatch = UserWatch[UserWatch["Rating"] > minimum_user_interaction]

rating = rating[rating["UserID"].isin(UserWatch.index)]

In [13]:
rating_table = rating.pivot_table(index="MovieID", columns="UserID", values="Rating")

SparseRate(rating_table)

90.396